In [1]:
import osmnx as ox
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import folium
from shapely.geometry import LineString
from shapely.geometry import Point
from shapely.geometry import Polygon
from geopy.geocoders import Nominatim
from geopy.point import Point

In [2]:
# ignore ssl cert verification

ox.settings.requests_kwargs={
    'verify':False
}

#ignore warnings

import warnings
warnings.filterwarnings('ignore')

In [3]:
# reverse geocode lat/long

geolocator = Nominatim(user_agent="test")

def reverse_geocoding(lat, lon):
    try:
        location = geolocator.reverse(Point(lat, lon))
        return location.raw['display_name']
    except:
        return None

In [4]:
# assumed origin p&r locations and CBD destination

tepuna_site = [-37.68966554965317, 176.07276863384655]
omokoroa_site = [-37.66467993152612, 176.0164613282822]
tauriko_site = [-37.738500582474096, 176.09665898553007]
domain_site = [-37.713064967025396, 176.28392062649655]

pnr_sites = [tepuna_site, omokoroa_site, tauriko_site, domain_site]

cbd = [-37.68395662843406, 176.16732617871457]


In [5]:
# define grpah and mode

place = ["Tauranga City, New Zealand",
        "Western Bay of Plenty District, New Zealand"]

drive = "drive"
pt = "drive_service"

Gdr = ox.graph_from_place(place, network_type=drive)
Gpt = ox.graph_from_place(place, network_type=pt)
ox.add_edge_speeds(Gdr)
ox.add_edge_speeds(Gpt)
ox.add_edge_travel_times(Gdr)
ox.add_edge_travel_times(Gpt)

In [6]:
d = []
for site in pnr_sites:
    pnr_node = ox.nearest_nodes(Gdr, site[1], site[0])
    cbd_node = ox.nearest_nodes(Gdr, 176.16732617871457, -37.68395662843406)
    #edges = ox.graph_to_gdfs(G, edges=True, nodes=False)
    #edges["highway"] = edges["highway"].astype(str)
    #edges.groupby("highway")[["length", "speed_kph", "travel_time"]].mean().round(1)
    dr_route = ox.shortest_path(Gdr, pnr_node, cbd_node, weight="travel_time")
    dr_route_time = int(sum(ox.utils_graph.route_to_gdf(Gdr, dr_route, "travel_time")["travel_time"])) / 60 # sum route length in seconds of travel time and convert to minutes
    pt_route = ox.shortest_path(Gpt, pnr_node, cbd_node, weight="travel_time")
    pt_route_time = int(sum(ox.utils_graph.route_to_gdf(Gpt, pt_route, "travel_time")["travel_time"])) / 60 # sum route length in seconds of travel time and convert to minute
    result = {'Origin lat/long':site,
              'Origin name':np.vectorize(reverse_geocoding)(site[0],site[1]),
              'Destination name': np.vectorize(reverse_geocoding)(cbd[0],cbd[1]),
             'Driving Time (minutes)':dr_route_time,
             'PT Time (minutes)':pt_route_time}
    d.append(result)
df = pd.DataFrame(data=d)
df
    

,Origin lat/long,Origin name,Destination name,Driving Time (minutes),PT Time (minutes)
0,"[-37.68966554965317, 176.07276863384655]","Te Puna Road, Te Puna, Western Bay of Plenty D...","65, Durham Street, Tauranga Central, Tauranga,...",9.550000,9.516667
1,"[-37.66467993152612, 176.0164613282822]","Omokoroa Road, Western Bay of Plenty District,...","65, Durham Street, Tauranga Central, Tauranga,...",14.416667,14.383333
2,"[-37.738500582474096, 176.09665898553007]","764, State Highway 29, Tauriko, Tauranga, Taur...","65, Durham Street, Tauranga Central, Tauranga,...",6.900000,6.883333
3,"[-37.713064967025396, 176.28392062649655]","Domain Road, Tauranga, Tauranga City, Bay of P...","65, Durham Street, Tauranga Central, Tauranga,...",11.216667,11.266667
